### 1. Configurations

In [27]:
import os
os.environ["JAX_PLATFORM_NAME"] = "cpu"
import netket as nk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.offsetbox import AnchoredText
import json
import netket.nn as nknn
import flax.linen as nn
import jax.numpy as jnp
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import math 
import time

In [28]:
def graph(L):
    # Define custom graph
    edge_colors = []
    for i in range(L):
        edge_colors.append([i, (i+1)%L, 1])
        edge_colors.append([i, (i+2)%L, 2])
    # Define the netket graph object
    g = nk.graph.Graph(edges=edge_colors)    
    return g 

In [29]:
def bonds(J):
    #Sigma^z*Sigma^z interactions
    sigmaz = [[1, 0], [0, -1]]
    mszsz = (np.kron(sigmaz, sigmaz))

    #Exchange interactions
    exchange = np.asarray([[0, 0, 0, 0], [0, 0, 2, 0], [0, 2, 0, 0], [0, 0, 0, 0]])

    bond_operator = [
        (J[0] * mszsz).tolist(),
        (J[1] * mszsz).tolist(),
        (-J[0] * exchange).tolist(),  
        (J[1] * exchange).tolist(),
    ]

    bond_color = [1, 2, 1, 2]
    
    return bond_operator, bond_color

In [30]:
def operators(g,bond_operator,bond_color):
     
    # Spin based Hilbert Space
    hi = nk.hilbert.Spin(s=0.5, total_sz=0.0, N=g.n_nodes)
    
    # Custom Hamiltonian operator
    op = nk.operator.GraphOperator(hi, graph=g, bond_ops=bond_operator, bond_ops_colors=bond_color)
    
    return hi,op

In [42]:
def run(i,w,rad,J,size,net,n_it,alpha):
    
    PARAM                      = i
    
    g                          = graph(size)
    
    bond_operator, bond_color  = bonds(J)
     
    hi,op                      = operators(g,bond_operator,bond_color)
   
        
    ma = nk.models.RBM(alpha=alpha)

    # Build the sampler
    sa = nk.sampler.MetropolisExchange(hilbert=hi,graph=g)

    # Custom Hamiltonian operator
    op = nk.operator.GraphOperator(hi, graph=g, bond_ops=bond_operator, bond_ops_colors=bond_color)

    # Optimizer
    opt = nk.optimizer.Sgd(learning_rate=0.01)

    # Stochastic Reconfiguration
    sr = nk.optimizer.SR(diag_shift=0.1)

    # The variational state
    vs = nk.vqs.MCState(sa, ma, n_samples=2000)

    # The ground-state optimization loop
    gs = nk.VMC(
        hamiltonian=op,
        optimizer=opt,
        preconditioner=sr,
        variational_state=vs)

    # We need to specify the local operators as a matrix acting on a local Hilbert space 
    sf = []
    sites = []
    structure_factor = nk.operator.LocalOperator(hi, dtype=complex)
    for i in range(0, size):
        for j in range(0, size):
            structure_factor += (nk.operator.spin.sigmaz(hi, i)*nk.operator.spin.sigmaz(hi, j))*((-1)**(i-j))/size
      
    
    print('### RBM calculation')
    # Run the optimization protocol
    param_file ="log/" + str(PARAM) + "_" + str(size) + "_" + str(n_it) +  "_" + str(net)
    
    gs.run(out=param_file, n_iter=n_it, obs={'Structure Factor': structure_factor})

    data=json.load(open(param_file + ".log"))
    # Extract the relevant information
    iters_RBM = data["Energy"]["iters"]
    energy_RBM = data["Energy"]["Mean"]
    
    
    E_gs, ket_gs = nk.exact.lanczos_ed(op, compute_eigenvectors=True)
    structure_factor_gs = (ket_gs.T.conj()@structure_factor.to_linear_operator()@ket_gs).real[0,0]
    
    j1 = J[0]
    j2 = J[1]
    
    l = [PARAM,w,rad,j1,j2,structure_factor_gs,E_gs[0],iters_RBM[-1],energy_RBM[-1]]
    
    v = []
    
    v.append(l)
        
    df   = pd.DataFrame(v, columns=['i', 'w','rad','j1', 'j2', 'factor_e', 'exact_e_0', 'factor_c', 'calc_e_0'])
    
    param_file = "data/" + str(PARAM) + "_" + str(size) + "_" + str(n_it) + "_" + str(net) + ".csv"
    
    df.to_csv(param_file, index=None)
    
    print(df)
    

### 2. Run

The transition occurs at J2/J1, = 0.2411(1) (w = 0.2366 (Okamoto and Nomura 1992))

In [32]:
import numpy as np

In [33]:
import math  

In [34]:
w = 30
rad = math.radians(w)
(rad,math.sin(rad), 
 math.cos(rad), 
 math.tan(rad), 
 math.sin(rad)/math.cos(rad), 
 math.cos(rad)/math.sin(rad),
 1/math.tan(rad))

(0.5235987755982988,
 0.49999999999999994,
 0.8660254037844387,
 0.5773502691896257,
 0.5773502691896256,
 1.7320508075688776,
 1.7320508075688774)

In [35]:
math.atan(1/1.7320508075688774)

0.5235987755982988

In [36]:
math.degrees(math.atan(0.5773502691896257)) 

29.999999999999996

In [37]:
round(29.9999,2)

30.0

In [38]:
#j2/j1 = 0.2411

In [39]:
math.degrees(math.atan(1/0.2411))

76.44468880110787

In [40]:
w = 1/2  
math.atan(w)

0.4636476090008061

In [43]:
w     = 77
pi    = math.pi
size  = 6 
net   = 'RBM'
n_it  = 1200
alpha = 3 # RBM ansatz with alpha=10  
i     = 0

rad = math.radians(w)
sin = math.sin(rad)
cos = math.cos(rad)
j1  = sin
j2  = cos

J     = [j1,j2]
run(i,w,rad,J,size,net,n_it,1)

### RBM calculation


100%|█| 1200/1200 [00:20<00:00, 58.49it/s, Energy=-9.848036 ± 0.000011 [σ²=0.000


   i   w       rad       j1        j2  factor_e  exact_e_0  factor_c  calc_e_0
0  0  77  1.343904  0.97437  0.224951   3.00753  -9.848026      1199 -9.848036


In [44]:
w     = 70
limit = 90
pi    = math.pi
size  = 6 
net   = 'RBM'
n_it  = 1200
alpha = 5 # RBM ansatz with alpha=10

for i in range(w,limit):
    rad = math.radians(w)
    sin = math.sin(rad)
    cos = math.cos(rad)
    j1  = sin
    j2  = cos
    J     = [j1,j2]
    run(i,w,rad,J,size,net,n_it,1)
    w   = w + 1

### RBM calculation


100%|█| 1200/1200 [00:20<00:00, 58.22it/s, Energy=-8.953771 ± 0.000011 [σ²=0.000


    i   w      rad        j1       j2  factor_e  exact_e_0  factor_c  calc_e_0
0  70  70  1.22173  0.939693  0.34202  2.920382  -8.953768      1199 -8.953771
### RBM calculation


100%|█| 1200/1200 [00:21<00:00, 56.92it/s, Energy=-9.0868007 ± 0.0000092 [σ²=0.0


    i   w       rad        j1        j2  factor_e  exact_e_0  factor_c  \
0  71  71  1.239184  0.945519  0.325568  2.934938  -9.086815      1199   

   calc_e_0  
0 -9.086801  
### RBM calculation


...

    i   w       rad       j1        j2  factor_e  exact_e_0  factor_c  \
0  87  87  1.518436  0.99863  0.052336  3.090494 -10.936647      1199   

   calc_e_0  
0 -10.93664  
### RBM calculation


100%|█| 1200/1200 [32:13<00:00,  1.61s/it, Energy=-11.030992 ± 0.000028 [σ²=0.00


    i   w      rad        j1        j2  factor_e  exact_e_0  factor_c  \
0  88  88  1.53589  0.999391  0.034899  3.097035 -11.031003      1199   

    calc_e_0  
0 -11.030992  
### RBM calculation


100%|█| 1200/1200 [3:41:30<00:00, 11.08s/it, Energy=-11.122499 ± 0.000018 [σ²=0.


    i   w       rad        j1        j2  factor_e  exact_e_0  factor_c  \
0  89  89  1.553343  0.999848  0.017452  3.103332 -11.122504      1199   

    calc_e_0  
0 -11.122499  
